In [123]:
import pandas as pd
import numpy as np

## Get Data

In [2]:
dtrain = pd.read_csv('../data/1e7data_train.csv', index_col=0)
dtest = pd.read_csv('../data/1e7data_holdout.csv', index_col=0)
dtrain.head()

,Stable,Norbits_instability,Norbits,Mplanet,RH/a,a1,P1,e1,pomega1,inc1,...,norm_min_e2,norm_min_window10_e2,norm_min_e3,norm_min_window10_e3,norm_min_inc1,norm_min_window10_inc1,norm_min_inc2,norm_min_window10_inc2,norm_min_inc3,norm_min_window10_inc3
Sim_ID,,,,,,,,,,,,,,,,,,,,,
2414,True,1.000000e+07,10000000.0,0.000015,0.021544,0.056553,0.013449,0.008127,-4.462116,0.003942,...,0.000304,0.013332,0.000026,0.035966,0.000702,0.000327,0.000850,0.001853,0.000268,0.000527
4056,False,2.980163e+04,10000000.0,0.000015,0.021544,0.050793,0.011447,0.005201,4.606446,0.009269,...,0.000536,0.085972,0.000398,0.009833,0.000102,0.000159,0.000238,0.007656,0.000095,0.000473
4855,True,1.000000e+07,10000000.0,0.000015,0.021544,0.053113,0.012241,0.004883,0.605644,0.000564,...,0.000176,0.048310,0.000261,0.006232,0.000552,0.025484,0.000111,0.000176,0.000191,0.000011
517,True,1.000000e+07,10000000.0,0.000015,0.021544,0.056243,0.013338,0.008056,1.245784,0.007544,...,0.000050,0.004476,0.000432,0.000578,0.000763,0.000076,0.000403,0.002325,0.000358,0.000026
1520,False,2.241913e+04,10000000.0,0.000015,0.021544,0.048262,0.010603,0.012062,-0.577740,0.005068,...,0.000107,0.004912,0.000071,0.025597,0.000004,0.008796,0.000984,0.001798,0.000431,0.027389


In [8]:
#indices of runs
train_i = dtrain.index
test_i = dtest.index

In [61]:
columns = ['Time', 'Lyapunov Time']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]

sample = pd.read_csv("../data/featuresfromshortruns/output%d.csv"%train_i[0], names=columns)
del sample['Time']
sample['Lyapunov Time'][0] = 0
sample.head()

,Lyapunov Time,a1,P1,e1,pomega1,inc1,Omega1,f1,a2,P2,...,inc2,Omega2,f2,a3,P3,e3,pomega3,inc3,Omega3,f3
0,0.000000,0.056549,0.013447,0.008142,-4.467394,0.003942,-3.134681,1.954992,0.067501,0.017537,...,0.002068,-1.855341,1.465630,0.077645,0.021635,0.006589,1.988693,0.010630,2.969191,-1.678443
1,-0.202318,0.056536,0.013443,0.009014,-4.352241,0.003939,-3.136235,1.866199,0.067516,0.017543,...,0.002129,-1.862288,0.504250,0.077643,0.021634,0.007906,2.099365,0.010612,2.964521,-1.099823
2,-0.604674,0.056535,0.013442,0.007947,-4.281563,0.003936,-3.135819,1.817454,0.067537,0.017551,...,0.002136,-1.864697,-0.518746,0.077622,0.021626,0.007963,2.100301,0.010611,2.963802,-0.403497
3,-1.235684,0.056535,0.013442,0.007617,-4.455353,0.003935,-3.137396,2.014344,0.067512,0.017542,...,0.002161,-1.863708,4.695949,0.077650,0.021637,0.007356,2.281041,0.010608,2.962154,0.113696
4,-2.164848,0.056537,0.013443,0.008831,-4.436246,0.003929,-3.137588,2.020701,0.067508,0.017540,...,0.002236,-1.863911,3.582446,0.077653,0.021638,0.005705,2.100080,0.010604,2.955748,0.990928


In [124]:
#Each 1D image has length 2000 with 22 channels (or 22 colours)
X_train = []
y_train = []
for i in train_i[0:100]:
    sample = pd.read_csv("../data/featuresfromshortruns/output%d.csv"%i, names=columns)
    if len(sample) == 2000:
        del sample['Time']
        sample['Lyapunov Time'][0] = 0
        X_train.append(sample.values)
        y_train.append(int(dtrain['Stable'].iloc[dtrain.index == i]))
        
X_train = np.asarray(X_train)
y_train = np_utils.to_categorical(np.asarray(y_train))
num_classes = y_train.shape[1]

## Build Model

In [125]:
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Convolution1D, MaxPooling1D, Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils

In [132]:
#create model
model = Sequential()
model.add(Convolution1D(32, 100, input_shape=(22, 2000), border_mode='same', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_length=2))   #half the image in each direction
model.add(Convolution1D(32, 100,  activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling1D(pool_length=2))   #half the image in each direction
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [133]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_15 (Convolution1D) (None, 22, 32)        6400032     convolution1d_input_8[0][0]      
____________________________________________________________________________________________________
dropout_18 (Dropout)             (None, 22, 32)        0           convolution1d_15[0][0]           
____________________________________________________________________________________________________
maxpooling1d_12 (MaxPooling1D)   (None, 11, 32)        0           dropout_18[0][0]                 
____________________________________________________________________________________________________
convolution1d_16 (Convolution1D) (None, 11, 32)        102432      maxpooling1d_12[0][0]            
___________________________________________________________________________________________

In [134]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=32)

ValueError: Error when checking model input: expected convolution1d_input_8 to have shape (None, 22, 2000) but got array with shape (95, 2000, 22)

## Extra crap to be deleted later

In [22]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Untaring file...


In [52]:
print X_train.shape
print y_test.shape

(50000, 32, 32, 3)
(10000, 1)


In [53]:
#proper input to keras I think
y_train2 = np_utils.to_categorical(y_train)
print y_train2.shape
print y_train2[0]

(50000, 10)
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
